# **1 - Installation**

In [18]:
!pip install ../.. --upgrade --no-cache-dir

Processing /workspaces/codespaces-blank/nnetsauce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nnetsauce: filename=nnetsauce-0.18.0-py2.py3-none-any.whl size=164335 sha256=7cd43291880e64d0fabbdaf6be5163df08e775fa16d3bd64328f204658a8c610
  Stored in directory: /tmp/pip-ephem-wheel-cache-mh85jhpn/wheels/4e/ea/ca/cbcf65127122d720fabf8aac6bbf53e047e9a85cf8383d27f1
Successfully built nnetsauce
  Attempting uninstall: nnetsauce
    Found existing installation: nnetsauce 0.18.0
    Uninstalling nnetsauce-0.18.0:
      Successfully uninstalled nnetsauce-0.18.0


In [19]:
import os 
import nnetsauce as ns 
import numpy as np 
from sklearn.datasets import load_diabetes, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from time import time 

In [25]:
data = fetch_california_housing()
X = data.data
y= data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 213)
print(f"X_train.shape(): {X_train.shape}")
print(f"X_test.shape(): {X_test.shape}")

regr = ns.PredictionInterval(obj=ExtraTreesRegressor(),
                             type_pi="bootstrap",
                             replications=100, 
                             level=95,
                             seed=312)
start = time()
regr.fit(X_train, y_train)
print(f"Elapsed: {time() - start}s")
preds = regr.predict(X_test)
print(preds)
print(f"\n coverage_rate: {np.mean((preds[2]<=y_test)*(preds[3]>=y_test))} \n")

X_train.shape(): (14448, 8)
X_test.shape(): (6192, 8)
Elapsed: 1.4486358165740967s


100%|██████████| 100/100 [00:00<00:00, 61365.09it/s]

self.sims_.shape: (6192, 100)
DescribeResult(mean=array([3.2360117 , 1.24578213, 4.95591427, ..., 0.66750337, 1.56394379,
       0.56858288]), sims=array([[ 3.10472714e+00,  2.43864714e+00,  2.23507714e+00, ...,
         2.64397714e+00,  3.18869714e+00,  3.06951714e+00],
       [ 1.60569624e+00,  1.06447624e+00,  1.79409624e+00, ...,
         1.60569624e+00, -9.81846579e-02,  1.09772624e+00],
       [ 5.27418614e+00,  5.02272614e+00,  5.00947614e+00, ...,
         5.95603614e+00,  4.59848614e+00,  4.94088614e+00],
       ...,
       [ 6.10926242e-01,  3.80596242e-01,  9.11795842e-01, ...,
        -7.45705579e-02,  8.58416242e-01,  1.34840124e+00],
       [ 1.44491624e+00,  1.65474624e+00,  1.10092604e+00, ...,
         4.97054742e-01,  1.78303624e+00,  1.54335624e+00],
       [ 3.76614210e-03,  6.72984842e-01,  4.78606242e-01, ...,
        -4.77314558e-01,  3.26246242e-01,  3.03476242e-01]]), lower=array([ 2.2380006 , -0.0921739 ,  2.48888988, ..., -2.23056179,
        0.49958544, -1.2

In [26]:
import warnings

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np


split_color = 'green'
local_color = 'gray'

np.random.seed(1)

def plot_func(x,
              y,
              y_u=None,
              y_l=None,
              pred=None,
              shade_color="",
              method_name="",
              title=""):

    fig = plt.figure()

    plt.plot(x, y, 'k.', alpha=.3, markersize=10,
             fillstyle='full', label=u'Test set observations')

    if (y_u is not None) and (y_l is not None):
        plt.fill(np.concatenate([x, x[::-1]]),
                 np.concatenate([y_u, y_l[::-1]]),
                 alpha=.3, fc=shade_color, ec='None',
                 label = method_name + ' Prediction interval')

    if pred is not None:
        plt.plot(x, pred, 'k--', lw=2, alpha=0.9,
                 label=u'Predicted value')

    #plt.ylim([-2.5, 7])
    plt.xlabel('$X$')
    plt.ylabel('$Y$')
    plt.legend(loc='upper right')
    plt.title(title)

    plt.show()

In [27]:
max_idx = 50
plot_func(x = range(max_idx),
          y = y_test[0:max_idx],
          y_u = preds[3][0:max_idx],
          y_l = preds[2][0:max_idx],
          pred = preds[0][0:max_idx],
          shade_color=split_color,
          title = f"Split conformal ({max_idx} first points in test set)")